In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px


import warnings
warnings.filterwarnings('ignore')
pd.options.display.float_format = '{:.5f}'.format

In [ ]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [ ]:
# Google Drive Mount

from google.colab import auth
auth.authenticate_user()

from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
%cd /content/gdrive/Shareddrives/cj공모전/활용 데이터 및 코드/2. 시계열 클러스터링 코드/

/content/gdrive/Shareddrives/cj공모전/활용 데이터 및 코드/2. 시계열 클러스터링 코드


In [ ]:
%ls

 cluster_label.pkl                  spark-3.0.3-bin-hadoop2.7.tgz.1
 fc주문_통합_수정.pkl               spark-3.0.3-bin-hadoop2.7.tgz.2
 시계열클러스터링_최종_사본.ipynb   spark-3.0.3-bin-hadoop2.7.tgz.3
 시계열클러스터링_최종.ipynb        spark-3.0.3-bin-hadoop2.7.tgz.4
'시계열클러스터링 pre.ipynb'        spark-3.0.3-bin-hadoop2.7.tgz.5
 spark-3.0.3-bin-hadoop2.7/         spark-3.0.3-bin-hadoop2.7.tgz.6
 spark-3.0.3-bin-hadoop2.7.tgz      시계열week.pkl


In [ ]:
final_df =  pd.read_pickle('fc주문_통합_수정.pkl')

In [ ]:
tmp = final_df.groupby(['ITEM_CD']).sum()

In [ ]:
fig = px.box(tmp, y="ITEM_QTY")
fig.show()

In [ ]:
#아이템코드별로 수량을 집계 후 QTY의 박스플롯 Q1을 기준으로 7개 미만 아이템 코드는 drop
grouped = final_df.groupby('ITEM_CD').sum()
grouped = grouped['ITEM_QTY'].reset_index()
ilist = grouped[grouped['ITEM_QTY']>=7]['ITEM_CD']

In [ ]:
#주별 칼럼 만들기
final_df['week'] = final_df['BKG_DATE'].apply(lambda x: x.isocalendar()[1])

In [ ]:
final_df['week'] = final_df['week'].apply(lambda x: x-8)

In [ ]:
#아이템 코드, 주별로 수량 합계
final_df= final_df.groupby(['ITEM_CD','week'])['ITEM_QTY'].sum().reset_index()

In [ ]:
#주별 데이터 프레임 만들기
date_df = pd.DataFrame(range(1, 19))

In [ ]:
date_df.columns = ['BKG_DATE']

In [ ]:
item = pd.DataFrame(final_df.ITEM_CD.unique())
item.columns=['ITEM_CD']

In [ ]:
#7개 이상의 아이템 코드만 활용
item = item[item['ITEM_CD'].isin(ilist)]

In [ ]:
item.columns = ['ITEM_CD']

In [ ]:
item1 = item[:]

In [ ]:
date_id1 = pd.MultiIndex.from_product((date_df.BKG_DATE, item1.ITEM_CD))

In [ ]:
df1 = pd.DataFrame(index = date_id1)

In [ ]:
df1 = df1.reset_index()

In [ ]:
gb = final_df.groupby(['week','ITEM_CD']).sum()

In [ ]:
gb = gb[['ITEM_QTY']]

In [ ]:
gb = gb.reset_index()

In [ ]:
df1=df1.rename(columns={'BKG_DATE':'week'})

In [ ]:
#아이템코드와 주를 기준으로 조인
final1 = pd.merge(df1, gb, how = 'left', on=['week','ITEM_CD'])

In [ ]:
final1.fillna(0, inplace = True)

In [ ]:
final1=final1.astype({'ITEM_QTY':'int'})

In [ ]:
#아이템코드와 주를 기준으로 매트릭스 형성
pdf1 = pd.pivot_table(final1, index = 'ITEM_CD', columns = 'week', values= 'ITEM_QTY')

In [33]:
pdf1.fillna(0, inplace = True)

In [34]:
pdf1['ITEM_CD'] = pdf1.index

In [36]:
pdf1.to_pickle("시계열week.pkl")